In [7]:
import subprocess
import os
from dotenv import load_dotenv
import subprocess

# Load environment variables from .env file
load_dotenv("credentials.env")

# Helper function to run Azure CLI commands and return output
def run_az_command(command):
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error running command: {result.stderr}")
        return None
    return result.stdout.strip()

def get_search_service_info(resource_group, search_service_name):
    search_endpoint = run_az_command([
        "az", "search", "service", "show",
        "--resource-group", resource_group,
        "--name", search_service_name,
        "--query", "hostName", "-o", "tsv"
    ])
    search_key = run_az_command([
        "az", "search", "admin-key", "show",
        "--resource-group", resource_group,
        "--service-name", search_service_name,
        "--query", "primaryKey", "-o", "tsv"
    ])
    return search_endpoint, search_key

def get_openai_service_info(resource_group, openai_service_name):
    openai_endpoint = run_az_command([
        "az", "cognitiveservices", "account", "show",
        "--resource-group", resource_group,
        "--name", openai_service_name,
        "--query", "properties.endpoint", "-o", "tsv"
    ])
    openai_key = run_az_command([
        "az", "cognitiveservices", "account", "keys", "list",
        "--resource-group", resource_group,
        "--name", openai_service_name,
        "--query", "key1", "-o", "tsv"
    ])
    return openai_endpoint, openai_key

# Get storage account connection string
def get_storage_account_connection_string(resource_group, storage_account_name):
    command = [
        "az", "storage", "account", "show-connection-string",
        "--resource-group", resource_group,
        "--name", storage_account_name,
        "--query", "connectionString", "-o", "tsv"
    ]
    return run_az_command(command)

# Generate a storage account-level SAS token
def generate_storage_account_sas(storage_account_name):
    command = [
        "az", "storage", "account", "generate-sas",
        "--permissions", "rwdlacup",  # Read, Write, Delete, List, etc.
        "--account-name", storage_account_name,
        "--services", "b",  # Blob service
        "--resource-types", "sco",  # Service, Container, Object
        "--expiry", "2024-12-31T23:59:00Z",  # Customize expiry date
        "--https-only", "--output", "tsv"
    ]
    return run_az_command(command)

# Main function to get connection string and SAS token for account-level access
def get_storage_access_info(resource_group, storage_account_name):
    # Get storage account connection string
    connection_string = get_storage_account_connection_string(resource_group, storage_account_name)
    if not connection_string:
        print(f"Failed to get connection string for storage account: {storage_account_name}")
        return None
    
    # Generate storage account-level SAS token
    sas_token = generate_storage_account_sas(storage_account_name)
    if not sas_token:
        print(f"Failed to generate SAS token for storage account: {storage_account_name}")
        return None
    
    # Return both connection string and storage account-level SAS token
    return {
        "BLOB_CONNECTION_STRING": connection_string,
        "BLOB_SAS_TOKEN": sas_token
    }

def get_cosmosdb_info(resource_group, cosmosdb_account_name):
    cosmosdb_endpoint = run_az_command([
        "az", "cosmosdb", "show",
        "--resource-group", resource_group,
        "--name", cosmosdb_account_name,
        "--query", "documentEndpoint", "-o", "tsv"
    ])
    cosmosdb_connection_string = run_az_command([
        "az", "cosmosdb", "keys", "list",
        "--resource-group", resource_group,
        "--name", cosmosdb_account_name,
        "--type", "connection-strings",
        "--query", "connectionStrings[0].connectionString", "-o", "tsv"
    ])
    return cosmosdb_endpoint, cosmosdb_connection_string

def get_sql_info(resource_group, sql_server_name):
    sql_server = run_az_command([
        "az", "sql", "server", "show",
        "--resource-group", resource_group,
        "--name", sql_server_name,
        "--query", "fullyQualifiedDomainName", "-o", "tsv"
    ])
    # SQL username and password would typically be provided manually
    return sql_server

def get_bing_search_info(resource_group, bing_search_name):
    bing_subscription_key = run_az_command([
        "az", "cognitiveservices", "account", "keys", "list",
        "--resource-group", resource_group,
        "--name", bing_search_name,
        "--query", "key1", "-o", "tsv"
    ])
    return bing_subscription_key

# Main function to get all necessary information from Azure resources
def main():
    load_dotenv("credentials.env")


    resource_group = os.getenv("RESOURCE_GROUP")
    app_id = os.getenv("APPLICATIONID")
    BACKEND_SECRET = os.getenv("BACKEND_SECRET")
    blob_sas_token = os.getenv("BLOB_SAS_TOKEN")
    azure_search_name = os.getenv("COG_SERVICES_NAME")
    azure_openai_name = os.getenv("AZURE_OPENAI_NAME")
    azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
    sql_server_name = os.getenv("SQL_SERVER_NAME")
    sql_server_db = os.getenv("SQL_SERVER_DB")
    sql_server_user = os.getenv("SQL_SERVER_USERNAME")
    sql_server_password = os.getenv("SQL_SERVER_PASSWORD")
    cosmosdb_name = os.getenv("COSMOSDB_NAME")
    cosmosdb_container = os.getenv("COSMOSDB_CONTAINER")
    search_service_name = os.getenv("COG_SERVICES_NAME")
    openai_service_name = os.getenv("AZURE_OPENAI_NAME")
    storage_account_name = "myStorageAccount"
    container_name = "myContainer"
    cosmosdb_account_name = os.getenv("COSMOSDB_NAME")
    bing_search_name = os.getenv("BING_SUBSCRIPTION_KEY")

    # Get all resource information
    print("Getting information for Azure resources...")
    print("Resource Group:", resource_group)
    print ("Search_service_name: ", search_service_name)
    search_endpoint, search_key = get_search_service_info(resource_group, search_service_name)
    openai_endpoint, openai_key = get_openai_service_info(resource_group, openai_service_name)
    storage_access_info = get_storage_access_info(resource_group, storage_account_name)
    cosmosdb_endpoint, cosmosdb_connection_string = get_cosmosdb_info(resource_group, cosmosdb_account_name)
    sql_server = get_sql_info(resource_group, sql_server_name)
    bing_subscription_key = get_bing_search_info(resource_group, bing_search_name)

    # Print or store the information as needed
    print("AZURE_SEARCH_ENDPOINT:", search_endpoint)
    print("AZURE_SEARCH_KEY:", search_key)
    print("AZURE_OPENAI_ENDPOINT:", openai_endpoint)
    print("AZURE_OPENAI_API_KEY:", openai_key)
    print("BLOB_CONNECTION_STRING:", storage_access_info.get("BLOB_CONNECTION_STRING"))
    print("BLOB_SAS_TOKEN:", storage_access_info.get("BLOB_SAS_TOKEN"))
    print("AZURE_COSMOSDB_ENDPOINT:", cosmosdb_endpoint)
    print("AZURE_COSMOSDB_CONNECTION_STRING:", cosmosdb_connection_string)
    print("SQL_SERVER:", sql_server)
    print("BING_SUBSCRIPTION_KEY:", bing_subscription_key)

if __name__ == "__main__":
    main()


Getting information for Azure resources...
Resource Group: mighty-guinea-46743-southcentralus-rg
Search_service_name:  cognitive-service-ffpanhhmq7wy3
Error running command: ERROR: (ResourceNotFound) The Resource 'Microsoft.Search/searchServices/cognitive-service-ffpanhhmq7wy3' under resource group 'mighty-guinea-46743-southcentralus-rg' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Search/searchServices/cognitive-service-ffpanhhmq7wy3' under resource group 'mighty-guinea-46743-southcentralus-rg' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

Error running command: ERROR: (ResourceNotFound) The Resource 'Microsoft.Search/searchServices/cognitive-service-ffpanhhmq7wy3' under resource group 'mighty-guinea-46743-southcentralus-rg' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Mic

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [5]:
import os
import subprocess
import os
from dotenv import load_dotenv

load_dotenv("credentials.env")

# Check if environment variables are being accessed
print(f"AZURE_SEARCH_NAME: {os.getenv('AZURE_SEARCH_NAME')}")
print(f"BING_SUBSCRIPTION_KEY: {os.getenv('BING_SUBSCRIPTION_KEY')}")



Note: you may need to restart the kernel to use updated packages.
AZURE_SEARCH_NAME: cog-search-ffpanhhmq7wy3
BING_SUBSCRIPTION_KEY: ba57ac5b9e354bfe944aad7df551628e
